# Generation Notebook

## 2D Generation

### Utils functions

In [20]:
from typing import TypedDict

In [19]:
world = [
    [0,0,1,0,0],
    [0,1,1,0,0],
    [1,1,1,1,0],
    [0,0,1,1,0],
    [0,0,0,1,0]
    ]

In [48]:
posdict = dict()

In [51]:
def print_dict(d: dict):
    for key, value in d.items():
        print(f"{key} : {value}")

In [53]:
def get_close(world: list[list[int]], posdict: dict[tuple[int],list[tuple[int]]], pos: tuple[int], cell_type: int) -> dict[tuple[int],list[tuple[int]]]:
    
    posdict[pos] = []
    
    for x in range(pos[0]-1, pos[0]+2):
        if x == -1 or x == pos[0] or x >= len(world[0]):
            pass
        else : 
            if world[x][pos[1]] == cell_type: 
                posdict[pos].append((x,pos[1]))
                
    for y in range(pos[1]-1, pos[1]+2):
        if y == -1 or y == pos[1] or y >= len(world[0]):
            pass 
        else : 
            if world[pos[0]][y] == cell_type: 
                posdict[pos].append((pos[0],y))    
    
    return posdict

In [56]:
for line in range(len(world)):
    for column in range(len(world)): 
        if (line, column) not in posdict.keys():
            get_close(world, posdict, (line, column), 1)
        
print_dict(posdict)

(0, 0) : []
(0, 1) : [(1, 1), (0, 2)]
(0, 2) : [(1, 2)]
(0, 3) : [(0, 2)]
(0, 4) : []
(1, 0) : [(2, 0), (1, 1)]
(1, 1) : [(2, 1), (1, 2)]
(1, 2) : [(0, 2), (2, 2), (1, 1)]
(1, 3) : [(2, 3), (1, 2)]
(1, 4) : []
(2, 0) : [(2, 1)]
(2, 1) : [(1, 1), (2, 0), (2, 2)]
(2, 2) : [(1, 2), (3, 2), (2, 1), (2, 3)]
(2, 3) : [(3, 3), (2, 2)]
(2, 4) : [(2, 3)]
(3, 0) : [(2, 0)]
(3, 1) : [(2, 1), (3, 2)]
(3, 2) : [(2, 2), (3, 3)]
(3, 3) : [(2, 3), (4, 3), (3, 2)]
(3, 4) : [(3, 3)]
(4, 0) : []
(4, 1) : []
(4, 2) : [(3, 2), (4, 3)]
(4, 3) : [(3, 3)]
(4, 4) : [(4, 3)]


In [5]:
def get_biome_size(world: list[list[int]], biome_id: int) : 
    pass